In [1]:
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
# the benefit of setting environmental variable is to protect your api key when sharing code with others
# if you don't know how to set env, just copy your api key into the following code 
# and make sure to remove your key when sharing code with others

# openai.api_key = ""

In [3]:
def get_gpt_response(sys_msg, prompt, ex, max_tokens):
    response = openai.ChatCompletion.create(
        #model="gpt-3.5-turbo",
        model='gpt-4',
        messages=[
            {
                "role": "system",
                "content": sys_msg
            },
            {
                "role": "user",
                "content": prompt + "\n" + "'''" + ex + "'''"
            }
                ],
            temperature=0,
            max_tokens=max_tokens
    )
    gpt_answer = response['choices'][0]['message']['content'] 
    print("answer", gpt_answer)

In [4]:
def get_batch_responses(sys_msg, prompt, inputs, max_tokens):
    for ex in inputs:
        get_gpt_response(sys_msg, prompt, ex, max_tokens)

In [5]:
max_tokens = 50
sys_message = ""

# Category labels for citation purpose classification

Purpose – why an author evaluates the cited article(s);

four types of purpose - comparison, critique, use, and info.

Sometimes an author cites with an explicit evaluation purpose, such as comparison and critique. Sometimes the evaluation purpose is implicit; for example, if an author cites by saying “we used the method developed by A”, although there seems no evaluation, the author actually endorsed the method by adopting it. Despite such explicit and implicit evaluation, many citations do not show any cues for evaluation purpose. These citations should be annotated as info, indicating the purpose of the citation is to simply point readers to some reference(s) for further reading.

# sample test examples

In [6]:
import pandas as pd
critique_ex = pd.read_excel('data/cora-small-sample-critique.xlsx')
critique_ex.head()

,citation_context,purpose,aspect,sentiment
0,"For example, some investigators have argued co...",critique,method,4_negative
1,Several clinical criteria and decision tools h...,critique,method,4_negative
2,Importantly the traffic light system missed a ...,critique,method,4_negative
3,The NICE fever guidelines advise routine testi...,critique,method,4_positive
4,"In this context, one can view successful progr...",critique,method,4_positive


In [7]:
inputs = critique_ex['citation_context']

# intuitive manual prompt

In [8]:
# intuitive manual prompt
manual_prompt_1 = "What is the function of the citation in the sentence? \
    *Comparison*, *Critique*, *Use*, or *Info* (the priority of Info is the lowest)? \
    The sentence is:"

get_batch_responses(sys_message, manual_prompt_1, inputs, max_tokens)

answer *Use*
answer *Critique*
answer *Info*
answer *Use*
answer *Use*
answer *Use*
answer *Use*
answer *Info*
answer *Use*
answer *Use*


In [9]:
manual_prompt_2 = "Annotate the citation function in the following sentence as either \
    *Comparison*, *Critique*, *Use*, or *Info* (the priority of Info is the lowest): "

get_batch_responses(sys_message, manual_prompt_2, inputs, max_tokens)

answer *Use*
answer *Critique*
answer *Comparison*
answer *Use*
answer *Use*
answer *Critique*
answer *Use*
answer *Info*
answer *Use*
answer *Use*


In [10]:
# adding system message

sys_message="You are a librarian."

get_batch_responses(sys_message, manual_prompt_2, inputs, max_tokens)

answer *Critique*
answer *Critique*
answer The citation function in the sentence is *Comparison*.
answer *Use*
answer *Use*
answer *Critique*
answer *Use*
answer *Info*
answer *Use*
answer *Use*


In [31]:
# reset system message
sys_message = ""

# add more context

In [11]:
# adding definition of labels

context = "Sometimes an author cites with an explicit evaluation purpose,\
    such as comparison and critique. Sometimes the evaluation purpose is implicit;\
    for example, if an author cites by saying 'we used the method developed by A', \
    although there seems no evaluation, the author actually endorsed the method by adopting it.\
    Despite such explicit and implicit evaluation, many citations do not show any cues for evaluation purpose.\
    These citations should be annotated as *Info*, \
    indicating the purpose of the citation is to \
    simply point readers to some reference(s) for further reading."
    
context_prompt_1 = context + manual_prompt_2

get_batch_responses(sys_message, context_prompt_1, inputs, max_tokens)

answer *Critique*
answer *Critique*
answer *Critique*
answer *Use*
answer *Use*
answer *Critique*
answer *Use*
answer *Use*
answer *Use*
answer *Critique*


# add Chain-of-Thought

In [12]:
# adding CoT

CoT_prompt = "Think step by step"
CoT_prompt_2 = context_prompt_1 + CoT_prompt
get_batch_responses(sys_message, CoT_prompt_2, inputs, max_tokens)

answer *Critique*
answer *Critique*
answer *Critique*
answer *Use*
answer *Use*
answer *Critique*
answer *Comparison*
answer *Comparison*
answer *Use*
answer *Critique*


# learn how ChatGPT defines the labels

In [13]:
# ask chatGPT how it defines *Critique*

prompt = "How do you define *Critique* as the purpose of a citation"

get_gpt_response(sys_message, prompt, "", 200)

answer Critique as the purpose of a citation refers to the use of a citation to critically analyze, evaluate, or interpret the cited work. This could involve discussing the strengths and weaknesses of the work, comparing it to other works on the same topic, or providing a personal perspective or interpretation. The citation serves as a way to give credit to the original source of the ideas or information being critiqued, and also allows readers to locate the original work if they wish to explore it further.


In [14]:
# define *Critique* as "Critique as the purpose of a citation refers to the use of a citation to critically analyze, evaluate, or interpret the cited work. This could involve discussing the strengths and weaknesses of the work, comparing it to other works on the same topic, or providing a personal perspective or interpretation."
# use this definition as context

gpt_def_critique = "Critique as the purpose of a citation \
    refers to the use of a citation to critically analyze, \
    evaluate, or interpret the cited work. This could involve \
    discussing the strengths and weaknesses of the work, \
    comparing it to other works on the same topic, \
    or providing a personal perspective or interpretation."

context_prompt_2 = gpt_def_critique + manual_prompt_2

get_batch_responses(sys_message, context_prompt_2, inputs, max_tokens)

answer *Critique*
answer *Critique*
answer *Comparison*
answer *Use*
answer *Use*
answer *Critique*
answer *Critique*
answer *Comparison*
answer *Use*
answer *Use*


In [15]:
# remove the part related to *Comparison*
gpt_def_critique_2 = "Critique as the purpose of a citation \
    refers to the use of a citation to critically analyze, \
    evaluate, or interpret the cited work. This could involve \
    discussing the strengths and weaknesses of the work, \
    or providing a personal perspective or interpretation."

context_prompt_3 = gpt_def_critique_2 + manual_prompt_2

get_batch_responses(sys_message, context_prompt_3, inputs, max_tokens)

answer *Critique*
answer *Critique*
answer *Comparison*
answer *Use*
answer *Use*
answer *Critique*
answer *Critique*
answer *Use*
answer *Use*
answer *Use*
